In [1]:
import tweepy
from tkinter import *
import tkinter as tk
import tkintermapview as tkm
from geopy.geocoders import Nominatim
import pickle
import math

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("kerala_flood_nepal_earthquake_english_only.csv")

In [19]:
li = df["tweet"].to_list()

In [21]:
import random 
random.shuffle(li)

In [22]:
newlist = li[:100]

['Tremors felt in Delhi-NCR as 5.4 magnitude earthquake hits Nepal https://t.co/VZSODyi3As #Nepal',
 '@TonoSVJ Nepal...',
 '255\nANALYSIS: Misleading\n\nFACT: Shared video is an old video of 2015 from Kathmandu Nepal, being circulated in wrong context. Nothing to do with recent earthquake. \n\nINTENT: Users are sharing old video and connecting with recent earthquake without verifying news for clickbait. https://t.co/upMB1lWF6p',
 'Netflix doccie on the Nepal earthquake is heartbreaking',
 'RT @onlyrgu: Huge respect to Kerala Electricity Board for working in worst weather conditions on #KeralaFloods #KeralaFloodRelief',
 'Contributions to Chief ministers distress relief fund (CMDRF)crossed 1026 crore since august 30 #Keralafloods @CMOKerala @News18Kerala',
 'Praying for my kerala friends @imSanthoshP @rajKtomy . Hope you guys are safe. Praying for all Kerala all flood victim of people.Hope @PMOIndia give more support for them',
 'Grate victory for the team @CricketNamibia1 \nA strong ma

In [2]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC

In [3]:
# [13:51, 11/03/2023] Abhay: api key qkBxRRYAv1KGXLLQqg6FkzlP4
# [13:52, 11/03/2023] Abhay: api key secret 3rAK403gUYnKAmccEO2sQOKjnakjZuPS9uyQZ97UZuSrnalAUi
# [13:52, 11/03/2023] Abhay: bearer token AAAAAAAAAAAAAAAAAAAAAE2FhwEAAAAAlknGU863CmXhK7OCQLNZxnQtEZ0%3DE1OsLHbSOtLvPG4VQlWWO7WlOkJpxnzmMkRig0AuxIKIDQ1UGY
# [13:53, 11/03/2023] Abhay: access token 1574425806545117184-7A5qOM7dRe5UjCCuUVdzjbmuhOeICr
# [13:53, 11/03/2023] Abhay: access token secret lyz9sy828kUYX0c2429lce4eCjFJR0RHKx0189BczBczz

In [4]:
#fetch live tweets
import tweepy

# Variables that contains the credentials to access Twitter API
ACCESS_TOKEN = '1574425806545117184-7A5qOM7dRe5UjCCuUVdzjbmuhOeICr'
ACCESS_SECRET = 'lyz9sy828kUYX0c2429lce4eCjFJR0RHKx0189BczBczz'
CONSUMER_KEY = 'qkBxRRYAv1KGXLLQqg6FkzlP4'
CONSUMER_SECRET = '3rAK403gUYnKAmccEO2sQOKjnakjZuPS9uyQZ97UZuSrnalAUi'


# Setup access to API
def connect_to_twitter_OAuth():
    auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
    auth.set_access_token(ACCESS_TOKEN, ACCESS_SECRET)

    api = tweepy.API(auth)
    return api


# Create API object
api = connect_to_twitter_OAuth()

In [5]:
root_tk = tk.Tk()
root_tk.geometry(f"{800}x{600}")
root_tk.title("map_view.py")



''

In [6]:
model_ops_list = ["BERT", "SVC"]
value_inside = tk.StringVar(root_tk)# text in the select option
value_inside.set("Select a Model")
model_select = tk.StringVar(root_tk)# text in the select option
drop = tk.OptionMenu(root_tk, value_inside, *model_ops_list,
#                      relief = GROOVE
                    )
drop.grid(row=0, column=0)


In [7]:
def search_event(event=None):
    map_widget.set_address(search_bar.get())

In [8]:
search_bar = tk.Entry(root_tk, width=50, relief = GROOVE)
search_bar.grid(row=0, column=1, pady=10, padx=10, sticky="we")
search_bar.focus()

search_bar_button = tk.Button(root_tk, width=8, text="Search", command=search_event, relief = GROOVE)
search_bar_button.grid(row=0, column=2, pady=10, padx=10)

# search_bar_clear = tk.Button(root_tk, width=8, text="Clear", command=clear)
# search_bar_clear.grid(row=0, column=2, pady=10, padx=10)

In [9]:
# create map widget
map_widget = tkm.TkinterMapView(root_tk, width=800, height=600, corner_radius=0)
map_widget.place(relx=0.5, rely=0.5, anchor=tk.CENTER)
map_widget.grid(row=1, column=1, columnspan=2, sticky="nsew")
# tk.search_marker = None
# tk.search_in_progress = False

In [10]:
def define_poly(coords):
    deg = 0
    poly = []
    while deg < 2*math.pi:# in the form [()]
        poly.append((coords[0] + (0.01*math.cos(deg)), coords[1] + (0.01*math.sin(deg))))
        deg = deg + (2*math.pi/360)
    return poly

In [11]:
def add_marker_event(coords):
#     global to_be_classified
    print("Add marker:", coords)
    new_marker = map_widget.set_marker(coords[0], coords[1], text="new marker")
    poly = define_poly(coords)
#     print(new_marker.position, new_marker.text)  # get position and text
    to_be_classified = find_tweet(coords)
    pred_condition = prediction(to_be_classified)
    if pred_condition:
        polygon_1 = map_widget.set_polygon(poly, fill_color= "red", outline_color="red", name="disaster_affected_area")
    else:
        polygon_1 = map_widget.set_polygon(poly, fill_color= "green", outline_color="green", name="disaster_affected_area")
    
    print(f"\n\n {to_be_classified}")
    

    

map_widget.add_right_click_menu_command(label="Add Marker",
                                        command=add_marker_event,
                                        pass_coords=True)


In [12]:
# Initialize Nominatim API
def find_tweet(coords):
    to_be_classified = []#this is going to be passed ahead
#     coord_str = str(coords[0]) + ", " + str(coords[1])     
#     print("cordinates are: ", coords)
#     geolocator = Nominatim(user_agent="MyApp")
#     # Latitude & Longitude input
#     coordinates = coord_str
#     location = geolocator.reverse(coordinates)
#     address = location.raw['address']
#     # Traverse the data
#     city = address.get('city', '')
#     state = address.get('state', '')
#     country = address.get('country', '')
#     print(f"{city} \n{state} \n{country} \n")
#     places = api.search_geo(lat = coords[0], long = coords[1], granularity="city")
#     # for place in places:    
#     #     print("placeid:%s" % place)
#     placed = "placeid:%s" % places
#     #item iterator class 
#     public_tweets = tweepy.Cursor(api.search_tweets, count=5,q="place:%s" % places[0].id, tweet_mode="extended", lang = "en").items(50)                            
#     for tweet in public_tweets:
#         to_be_classified.append(tweet.full_text)
#         print("result of the find tweet function", tweet.full_text)
    
    return to_be_classified

In [13]:
def prediction(to_be_classified):
    print(value_inside.get())
    if value_inside.get() == "SVC":
        filename = '/Major_Project/models/text_clf.sav'
        loaded_model = pickle.load(open(filename, 'rb'))
    elif value_inside.get() == "BERT":
        import os
        os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
        import tensorflow as tf
        import tensorflow_hub as hub
        import tensorflow_text as text
        from sklearn.model_selection import train_test_split
        print('Tensorflow module imported')
        loaded_model = tf.keras.models.load_model('/Major_Project/models/bert_success_1/bert_success_1')
    else:
        filename = '/Major_Project/models/text_clf.sav'
        loaded_model = pickle.load(open(filename, 'rb'))
        
    predictions = loaded_model.predict(to_be_classified)
    print(f"\n\n {loaded_model.predict(to_be_classified)}")
    disaster = 0 #count number of disaster events
    for i in predictions:
        if i:
            disaster +=1
    dis_cond = disaster / len(predictions)
    if dis_cond > 0.5:
        return 1
    else:
        return 0


In [ ]:
root_tk.mainloop()

Add marker: (52.51668365952665, 13.363543665237415)
cordinates are:  (52.51668365952665, 13.363543665237415)
Berlin 
 
Deutschland 

result of the find tweet function @bernard_prof @OUPAcademic Wow, congrats! Waiting for it!
result of the find tweet function moni’s angels 🤍 @ShawnMendes @ShawnAccess @TommyHilfiger + wincent &amp; tim #TommyXShawn https://t.co/m3JHQrLl3M
result of the find tweet function @germany_iam I live we get the world of spices! Gewürzewelt indeed!
result of the find tweet function Senior #SEO Manager (m/w/d) #HomeofficeFernarbeit (Fest) #Google #Content #FINELegal #dasauge #jobs → https://t.co/OYbGYYPPb3
result of the find tweet function @kiligsounds That's huge!!! Big congrats mate 🥳🥳❤️❤️
result of the find tweet function National TV who’s your editor. https://t.co/0e4IyLqPb9 https://t.co/e6CXXiqXFa
result of the find tweet function Sorry for the typos…. Tweeting before finishing coffee = bad idea ☕️
result of the find tweet function I am growing concerned for N

C:\Users\abhi1\anaconda3\lib\site-packages\sklearn\base.py:299: UserWarning: Trying to unpickle estimator TfidfTransformer from version 1.0.2 when using version 1.2.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\abhi1\anaconda3\lib\site-packages\sklearn\base.py:299: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 1.0.2 when using version 1.2.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\abhi1\anaconda3\lib\site-packages\sklearn\base.py:299: UserWarning: Trying to unpickle estimator LinearSVC from version 1.0.2 when using version 1.2.1. This might lead to breaking code or invalid results. Use at your own risk.

Add marker: (52.50383305751872, 13.394786035842884)
cordinates are:  (52.50383305751872, 13.394786035842884)
Berlin 
 
Deutschland 

result of the find tweet function Did I read the most insane fanfiction with dom/sub dynamics where Levi is the one obedient receiving degrading and CBT while Hange goes feral from their intense power play need? Alexa, hold my beer and play my masturbation playlist.
result of the find tweet function @Ryanair customer support is the worst I've experienced in my life. Stating this objectively and without exaggeration.

Quoting me back emails that don't exist, telling me to "just double check", ending the chat when I don't reply immediately 👎
result of the find tweet function @bernard_prof @OUPAcademic Wow, congrats! Waiting for it!
result of the find tweet function moni’s angels 🤍 @ShawnMendes @ShawnAccess @TommyHilfiger + wincent &amp; tim #TommyXShawn https://t.co/m3JHQrLl3M
result of the find tweet function @germany_iam I live we get the world of spices!

Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\abhi1\anaconda3\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
  File "C:\Users\abhi1\AppData\Local\Temp\ipykernel_2296\1468744194.py", line 8, in add_marker_event
    pred_condition = prediction(to_be_classified)
  File "C:\Users\abhi1\AppData\Local\Temp\ipykernel_2296\177946943.py", line 11, in prediction
    import tensorflow_text as text
  File "C:\Users\abhi1\AppData\Roaming\Python\Python39\site-packages\tensorflow_text\__init__.py", line 21, in <module>
    from tensorflow_text.python import keras
  File "C:\Users\abhi1\AppData\Roaming\Python\Python39\site-packages\tensorflow_text\python\keras\__init__.py", line 21, in <module>
    from tensorflow_text.python.keras.layers import *
  File "C:\Users\abhi1\AppData\Roaming\Python\Python39\site-packages\tensorflow_text\python\keras\layers\__init__.py", line 22, in <module>
    from tensorflow_text.python.keras.layers.tokeniz